# Cartpole Game

Deep-Q-Learning with Keras and Gym, based on [Keon Kim's](https://github.com/keon/deep-q-learning) code.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

#### Import dependencies

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

Using TensorFlow backend.


#### Set hyperparameters

In [3]:
output_dir = 'model_output/cartpole/'

n_episodes = 1001 # n games we want agent to play (default 1001)

#### Define class for Deep-Q-Learning agent

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95    # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0  # exploration rate: how much to act randomly; more initially than later due to decay
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance improves
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### Set other parameters (some of these should be moved to top of file)

In [5]:
env = gym.make('CartPole-v1') # initialise environment
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size) # initialise agent
# agent.load("./save/cartpole-dqn.h5") # JK FIX
done = False
batch_size = 32

In [6]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [7]:
for e in range(n_episodes): # iterate over new episodes of the game
    state = env.reset() # reset state at start of each new episode of the game
    state = np.reshape(state, [1, state_size])
    for time in range(5000): # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
#         env.render()
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position
        reward = reward if not done else -10 reward +1 for each additional frame with pole upright
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.
        state = next_state # set "current state" for upcoming iteration to the current next state
        if done: # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score
                  .format(e, n_episodes, time, agent.epsilon))
            break # exit loop
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5") 

episode: 0/1000, score: 8, e: 1.0
episode: 1/1000, score: 30, e: 1.0
episode: 2/1000, score: 12, e: 0.99
episode: 3/1000, score: 15, e: 0.99
episode: 4/1000, score: 30, e: 0.99
episode: 5/1000, score: 15, e: 0.98
episode: 6/1000, score: 14, e: 0.98
episode: 7/1000, score: 23, e: 0.97
episode: 8/1000, score: 24, e: 0.97
episode: 9/1000, score: 13, e: 0.96
episode: 10/1000, score: 22, e: 0.96
episode: 11/1000, score: 50, e: 0.95
episode: 12/1000, score: 35, e: 0.95
episode: 13/1000, score: 31, e: 0.94
episode: 14/1000, score: 15, e: 0.94
episode: 15/1000, score: 12, e: 0.93
episode: 16/1000, score: 17, e: 0.93
episode: 17/1000, score: 11, e: 0.92
episode: 18/1000, score: 48, e: 0.92
episode: 19/1000, score: 11, e: 0.91
episode: 20/1000, score: 11, e: 0.91
episode: 21/1000, score: 19, e: 0.9
episode: 22/1000, score: 11, e: 0.9
episode: 23/1000, score: 17, e: 0.9
episode: 24/1000, score: 46, e: 0.89
episode: 25/1000, score: 27, e: 0.89
episode: 26/1000, score: 17, e: 0.88
episode: 27/1000,

episode: 221/1000, score: 47, e: 0.33
episode: 222/1000, score: 62, e: 0.33
episode: 223/1000, score: 42, e: 0.33
episode: 224/1000, score: 34, e: 0.33
episode: 225/1000, score: 44, e: 0.33
episode: 226/1000, score: 52, e: 0.32
episode: 227/1000, score: 103, e: 0.32
episode: 228/1000, score: 161, e: 0.32
episode: 229/1000, score: 99, e: 0.32
episode: 230/1000, score: 92, e: 0.32
episode: 231/1000, score: 45, e: 0.32
episode: 232/1000, score: 83, e: 0.31
episode: 233/1000, score: 55, e: 0.31
episode: 234/1000, score: 63, e: 0.31
episode: 235/1000, score: 74, e: 0.31
episode: 236/1000, score: 157, e: 0.31
episode: 237/1000, score: 61, e: 0.31
episode: 238/1000, score: 169, e: 0.3
episode: 239/1000, score: 68, e: 0.3
episode: 240/1000, score: 64, e: 0.3
episode: 241/1000, score: 178, e: 0.3
episode: 242/1000, score: 35, e: 0.3
episode: 243/1000, score: 36, e: 0.3
episode: 244/1000, score: 38, e: 0.3
episode: 245/1000, score: 34, e: 0.29
episode: 246/1000, score: 20, e: 0.29
episode: 247/1

episode: 435/1000, score: 161, e: 0.11
episode: 436/1000, score: 162, e: 0.11
episode: 437/1000, score: 167, e: 0.11
episode: 438/1000, score: 203, e: 0.11
episode: 439/1000, score: 191, e: 0.11
episode: 440/1000, score: 133, e: 0.11
episode: 441/1000, score: 93, e: 0.11
episode: 442/1000, score: 91, e: 0.11
episode: 443/1000, score: 116, e: 0.11
episode: 444/1000, score: 118, e: 0.11
episode: 445/1000, score: 140, e: 0.11
episode: 446/1000, score: 141, e: 0.11
episode: 447/1000, score: 182, e: 0.11
episode: 448/1000, score: 14, e: 0.11
episode: 449/1000, score: 99, e: 0.11
episode: 450/1000, score: 37, e: 0.11
episode: 451/1000, score: 113, e: 0.1
episode: 452/1000, score: 129, e: 0.1
episode: 453/1000, score: 120, e: 0.1
episode: 454/1000, score: 234, e: 0.1
episode: 455/1000, score: 135, e: 0.1
episode: 456/1000, score: 499, e: 0.1
episode: 457/1000, score: 144, e: 0.1
episode: 458/1000, score: 142, e: 0.1
episode: 459/1000, score: 190, e: 0.1
episode: 460/1000, score: 187, e: 0.1
e

episode: 643/1000, score: 371, e: 0.04
episode: 644/1000, score: 239, e: 0.04
episode: 645/1000, score: 210, e: 0.04
episode: 646/1000, score: 232, e: 0.039
episode: 647/1000, score: 233, e: 0.039
episode: 648/1000, score: 269, e: 0.039
episode: 649/1000, score: 274, e: 0.039
episode: 650/1000, score: 220, e: 0.039
episode: 651/1000, score: 207, e: 0.038
episode: 652/1000, score: 209, e: 0.038
episode: 653/1000, score: 130, e: 0.038
episode: 654/1000, score: 160, e: 0.038
episode: 655/1000, score: 482, e: 0.038
episode: 656/1000, score: 499, e: 0.038
episode: 657/1000, score: 349, e: 0.037
episode: 658/1000, score: 267, e: 0.037
episode: 659/1000, score: 227, e: 0.037
episode: 660/1000, score: 224, e: 0.037
episode: 661/1000, score: 223, e: 0.037
episode: 662/1000, score: 216, e: 0.036
episode: 663/1000, score: 248, e: 0.036
episode: 664/1000, score: 258, e: 0.036
episode: 665/1000, score: 202, e: 0.036
episode: 666/1000, score: 229, e: 0.036
episode: 667/1000, score: 499, e: 0.035
epi

episode: 851/1000, score: 186, e: 0.014
episode: 852/1000, score: 85, e: 0.014
episode: 853/1000, score: 33, e: 0.014
episode: 854/1000, score: 51, e: 0.014
episode: 855/1000, score: 47, e: 0.014
episode: 856/1000, score: 54, e: 0.014
episode: 857/1000, score: 130, e: 0.014
episode: 858/1000, score: 63, e: 0.014
episode: 859/1000, score: 338, e: 0.014
episode: 860/1000, score: 208, e: 0.013
episode: 861/1000, score: 116, e: 0.013
episode: 862/1000, score: 132, e: 0.013
episode: 863/1000, score: 41, e: 0.013
episode: 864/1000, score: 25, e: 0.013
episode: 865/1000, score: 27, e: 0.013
episode: 866/1000, score: 37, e: 0.013
episode: 867/1000, score: 118, e: 0.013
episode: 868/1000, score: 129, e: 0.013
episode: 869/1000, score: 150, e: 0.013
episode: 870/1000, score: 117, e: 0.013
episode: 871/1000, score: 129, e: 0.013
episode: 872/1000, score: 132, e: 0.013
episode: 873/1000, score: 110, e: 0.013
episode: 874/1000, score: 134, e: 0.013
episode: 875/1000, score: 138, e: 0.013
episode: 8